# Active thermographic inspection for aviation asstes

In [1]:
# Dependencies
# pip install opencv-python bagpy cvbridge3 ipdb

## Read .bag thermal files

In [2]:
import rosbag
import os
import tqdm
import numpy as np
from sensor_msgs import point_cloud2

# Directory containing the bag files
main_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/"

# Create an output directory for saving overlayed images
output_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/output/"
os.makedirs(output_directory, exist_ok=True)

res = []

for root, dirs, files in os.walk(main_directory):
    for file in files:
        if not file.startswith('imu_gps'): # exclude gps bag files
            if file.endswith(".bag"):
                res.append(os.path.join(root, file))
print(res)

for file in res:

    bag             = rosbag.Bag(file)
    print (' - filename          : ', bag.filename)
    print (' - get_message_count : ', bag.get_message_count())
    print (' - start_time        : ', bag.get_start_time())
    print (' - end_time          : ', bag.get_end_time())
    print (' - size              : ', bag.size)
    print (' - mode              : ', bag.mode)
    print ('\n\n')

    ## Step2 - Method1 - Get all topics
    topics          = []
    print (' - get_message_count : ', bag.get_message_count())
    with tqdm.tqdm_notebook(total = bag.get_message_count()) as pbar:
        for idx, (topic, msg, t) in enumerate(bag.read_messages()):
            pbar.update(1)
            topics.append(topic)
    topics = np.array(topics)
    print (' - topics : ', np.unique(topics))

    ## Step2 - Method2 - Get all topics
    topics = [topic for topic in bag.get_type_and_topic_info().topics]
    for topic in sorted(topics):
        print (topic)

    ## Step3 - Print messages of a topic
    # for idx, (topic, msg, t) in enumerate(bag.read_messages()):
    for idx, (topic, msg, t) in enumerate(bag.read_messages(topics=['/velodyne_packets'])):
        print (topic, type(msg), t.to_time())

['/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_left_2023-08-31-14-53-51.bag', '/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_right_2023-08-31-14-53-51.bag']
 - filename          :  /Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_left_2023-08-31-14-53-51.bag
 - get_message_count :  352
 - start_time        :  1693508034.7076309
 - end_time          :  1693508060.4848423
 - size              :  230778409
 - mode              :  r



 - get_message_count :  352


/var/folders/df/rc0lymgs6nvd3r8bhd0wfb880000gn/T/ipykernel_24984/2356858404.py:37: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  with tqdm.tqdm_notebook(total = bag.get_message_count()) as pbar:


  0%|          | 0/352 [00:00<?, ?it/s]

 - topics :  ['/thermal_left/image']
/thermal_left/image
 - filename          :  /Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_right_2023-08-31-14-53-51.bag
 - get_message_count :  346
 - start_time        :  1693508034.864173
 - end_time          :  1693508060.4376736
 - size              :  226845181
 - mode              :  r



 - get_message_count :  346


  0%|          | 0/346 [00:00<?, ?it/s]

 - topics :  ['/thermal_right/image']
/thermal_right/image


In [32]:
import os
import cv2
import numpy as np
import rosbag
from sensor_msgs.msg import Image
from cv_bridge import CvBridge

# Initialize a CvBridge to convert ROS Image messages to OpenCV images
bridge = CvBridge()

# Directory containing the bag files
main_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/"

# Create an output directory for saving overlayed images
output_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/output/"
os.makedirs(output_directory, exist_ok=True)

# Function to process a single bag file
def process_bag_file(bag_file_path, main_directory, output_subdirectory):
    # Open the bag file
    bag = rosbag.Bag(bag_file_path)
    
    # Iterate through messages in the bag file
    for topic, msg, t in bag.read_messages(topics=['/thermal_right/image, /thermal_left/image']):
        process_message(msg, output_directory, output_subdirectory)

    # Close the bag file
    bag.close()

# Define a function to process each message in the bag file
def process_message(msg, main_directory, output_subdirectory):
    try:
        # Convert the ROS Image message to an OpenCV image
        cv_image = bridge.imgmsg_to_cv2(msg, desired_encoding="passthrough")

        # Apply any necessary preprocessing to enhance thermal features
        # For example, you can apply thresholding or other image processing techniques

        # Create a mask to isolate the thermal region of interest
        # Here, we assume the thermal region is in a specific temperature range
        lower_temp_threshold = 30  # Adjust this threshold as needed
        upper_temp_threshold = 100  # Adjust this threshold as needed
        thermal_mask = cv2.inRange(cv_image, lower_temp_threshold, upper_temp_threshold)

        # Apply the mask to the original image
        masked_image = cv2.bitwise_and(cv_image, cv_image, mask=thermal_mask)

        # Overlay the mask on the original image (for visualization)
        overlay_image = cv2.addWeighted(cv_image, 0.7, masked_image, 0.3, 0)

        # Save the overlayed image to the output subdirectory
        ipdb.set_trace()
        output_path = os.path.join(output_directory, output_subdirectory)
        os.makedirs(output_path, exist_ok=True)
        filename = os.path.splitext(os.path.basename(bag_file_path))[0] + "_overlayed.png"
        output_file_path = os.path.join(output_path, filename)
        cv2.imwrite(output_file_path, overlay_image)

    except Exception as e:
        print(e)

In [35]:
# Walk through the main directory and its subdirectories
import ipdb

for file in res:
    bag = rosbag.Bag(file)
    
    print(f"Processing {bag}...")
    process_bag_file(file, main_directory, output_directory)


# for root, dirs, files in os.walk(main_directory):
#     for file in files:
#         if not file.startswith('imu_gps'): # exclude GPS bag files
#             if file.endswith(".bag"):
                
# #                 ipdb.set_trace()
#                 # Get the relative subdirectory structure
#                 rel_subdirectory = os.path.relpath(root, main_directory)
#                 output_subdirectory = os.path.join(rel_subdirectory, file)

#                 print(f"Processing {bag_file_path}...")
#                 process_bag_file(bag_file_path, output_subdirectory)


Processing path:        /Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_left_2023-08-31-14-53-51.bag
version:     2.0
duration:    25.8s
start:       Aug 31 2023 14:53:54.71 (1693508034.71)
end:         Aug 31 2023 14:54:20.48 (1693508060.48)
size:        220.1 MB
messages:    352
compression: none [176/176 chunks]
types:       sensor_msgs/Image [060021388200f6f0f447d0fcd9c64743]
topics:      /thermal_left/image   352 msgs @ 14.3 Hz : sensor_msgs/Image...
Processing path:        /Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/after/thermal_right_2023-08-31-14-53-51.bag
version:     2.0
duration:    25.6s
start:       Aug 31 2023 14:53:54.86 (1693508034.86)
end:         Aug 31 2023 14:54:20.44 (1693508060.44)
size:        216.3 MB
messages:    346
compression: none [173/173 chunks]
types:       sensor_msgs/Image [060021388200f6f0f447d0fcd9c64743]
topics:      /thermal_right/image   346 msgs @ 14.3 Hz : sensor_msgs/Image...


In [19]:
import os
import cv2
import numpy as np

# Function to process and save masked images and masks separately
def process_images(input_dir, output_dir):
    
    # Iterate through each folder in the input directory
    for folder_name in os.listdir(input_dir):
        folder_path = os.path.join(input_dir, folder_name)
        if not os.path.isdir(folder_path):
            continue

        # Create the corresponding output folders
        masked_images_output = os.path.join(output_dir, folder_name, "masked_images")
        masks_output = os.path.join(output_dir, folder_name, "masks")
        os.makedirs(masked_images_output, exist_ok=True)
        os.makedirs(masks_output, exist_ok=True)

        # Iterate through images and masks in the folder
        
        
        for filename in os.listdir(folder_path):
            if filename.endswith('.bag'):
                image_path = os.path.join(folder_path, filename)
                mask_path = os.path.join(folder_path, filename.replace('.jpg', '_mask.jpg'))
                
                # Check if the mask file exists
                if os.path.exists(mask_path):
                    # Load the thermal image and mask
                    image = cv2.imread(image_path)
                    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)

                    # Apply the mask to the image
                    masked_image = cv2.bitwise_and(image, image, mask=mask)

                    # Save the masked image and mask separately
                    masked_image_output_path = os.path.join(masked_images_output, filename)
                    mask_output_path = os.path.join(masks_output, filename.replace('.jpg', '_mask.jpg'))

                    cv2.imwrite(masked_image_output_path, masked_image)
                    cv2.imwrite(mask_output_path, mask)

                    print(f"Processed: {masked_image_output_path} and {mask_output_path}")

# Define the input and output directories
input_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/"  # Replace with your input directory path
output_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/output/" # Replace with your output directory path

# Call the process_images function
process_images(input_directory, output_directory)


error: OpenCV(4.8.0) /Users/runner/work/opencv-python/opencv-python/opencv/modules/imgcodecs/src/loadsave.cpp:787: error: (-215:Assertion failed) !_img.empty() in function 'imwrite'


In [20]:
def extract_and_save_masks(bag_file, output_dir):
    bag = rosbag.Bag(bag_file)
    bridge = CvBridge()

    # Create an output directory for each Bag file
    bag_name = os.path.splitext(os.path.basename(bag_file))[0]
    bag_output_dir = os.path.join(output_dir, bag_name, 'masks')
    os.makedirs(bag_output_dir, exist_ok=True)

    for topic, msg, _ in bag.read_messages():
        if topic == '/thermal_camera/mask':
            # Convert the ROS Image message to a NumPy array
            mask = bridge.imgmsg_to_cv2(msg, desired_encoding="passthrough")

            # Save the mask image
            mask_filename = os.path.join(bag_output_dir, f'{msg.header.stamp}_mask.png')
            cv2.imwrite(mask_filename, mask)
            print(f'Saved mask: {mask_filename}')

    bag.close()

In [22]:
main_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/"
output_directory = "/Users/asghar/Desktop/AirLab/thermographic_inspection/2023-08-31/output/"

# Process each Bag file in the specified directory
for bag_filename in os.listdir(main_directory):
    if bag_filename.endswith('.bag'):
        bag_path = os.path.join(bag_directory, bag_filename)
        extract_and_save_masks(bag_path, output_directory)